In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind_from_stats
import scipy.stats as stats
from scipy.stats import norm, binom, beta
import math
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
#Import Datasets of demography
demo11_12 = pd.read_sas('/Users/Sylvia/Desktop/DEMO_G.XPT', encoding = 'utf-8')
demo13_14 = pd.read_sas('/Users/Sylvia/Desktop/DEMO_H.XPT', encoding = 'utf-8')
demo15_16 = pd.read_sas('/Users/Sylvia/Desktop/DEMO_I.XPT', encoding = 'utf-8')
demo17_18 = pd.read_sas('/Users/Sylvia/Desktop/DEMO_J.XPT', encoding = 'utf-8')

#Extract columns
updateddemo11_12 = demo11_12[['SEQN','RIDAGEYR','RIAGENDR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR','SDMVPSU','SDMVSTRA',
                             'WTMEC2YR','WTINT2YR']]
updateddemo13_14 = demo13_14[['SEQN','RIDAGEYR','RIAGENDR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR','SDMVPSU','SDMVSTRA',
                             'WTMEC2YR','WTINT2YR']]
updateddemo15_16 = demo15_16[['SEQN','RIDAGEYR','RIAGENDR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR','SDMVPSU','SDMVSTRA',
                             'WTMEC2YR','WTINT2YR']]
updateddemo17_18 = demo17_18[['SEQN','RIDAGEYR','RIAGENDR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR','SDMVPSU','SDMVSTRA',
                             'WTMEC2YR','WTINT2YR']]

#Create Cohorts
updateddemo11_12['cohort'] = '2011-2012'
updateddemo13_14['cohort'] = '2013-2014'
updateddemo15_16['cohort'] = '2015-2016'
updateddemo17_18['cohort'] = '2017-2018'

#Clean data
demo = pd.concat([updateddemo11_12, updateddemo13_14, updateddemo15_16, updateddemo17_18], axis = 0)
demo = demo.rename(columns ={'SEQN':'id','RIDAGEYR':'age','RIAGENDR':'gender','RIDRETH3':'race',
                             'DMDEDUC2':'education',
                            'DMDMARTL':'martial_status','RIDSTATR':'exam_status','SDMVPSU':'maskedvar-psu',
                            'SDMVSTRA':'maskedvar-stra','WTMEC2YR':'fullsample2yr_mec',
                            'WTINT2YR':'fullsample2yr_int'})
demo[['id','age','race']] = demo[['id','age','race']].astype(int)
demo[['education', 'martial_status', 'exam_status']] = demo[['education', 'martial_status', 'exam_status']].astype('category')
demo[['fullsample2yr_mec','fullsample2yr_int']] = demo[['fullsample2yr_mec','fullsample2yr_int']].round(2)
demo['fullsample2yr_mec'] = pd.to_numeric(demo['fullsample2yr_mec'], errors='coerce')
demo['maskedvar-psu'] = pd.to_numeric(demo['maskedvar-psu'], errors='coerce')
demo['maskedvar-stra'] = pd.to_numeric(demo['maskedvar-stra'], errors='coerce')
demo['cohort'] = demo['cohort'].astype('category')

#Import datasets of oral health
oral11_12 = pd.read_sas('/Users/Sylvia/Desktop/OHXDEN_G.XPT', encoding = 'utf-8')
oral13_14 = pd.read_sas('/Users/Sylvia/Desktop/OHXDEN_H.XPT', encoding = 'utf-8')
oral15_16 = pd.read_sas('/Users/Sylvia/Desktop/OHXDEN_I.XPT', encoding = 'utf-8')
oral17_18 = pd.read_sas('/Users/Sylvia/Desktop/OHXDEN_J.XPT', encoding = 'utf-8')

#Extract columns
updatedoral11_12 = oral11_12[['SEQN','OHDDESTS']]
updatedoral13_14 = oral13_14[['SEQN','OHDDESTS']]
updatedoral15_16 = oral15_16[['SEQN','OHDDESTS']]
updatedoral17_18 = oral17_18[['SEQN','OHDDESTS']]

#Setup cohorts
updatedoral11_12['cohort'] = '2011-2012'
updatedoral13_14['cohort'] = '2013-2014'
updatedoral15_16['cohort'] = '2015-2016'
updatedoral17_18['cohort'] = '2017-2018'

#Concat datasets
oral = pd.concat([updatedoral11_12, updatedoral13_14, updatedoral15_16, updatedoral17_18], axis = 0)
oral = oral.rename(columns = {'SEQN':'id', 'OHDDESTS':'ohx_status'})
oral[['id','ohx_status']] = oral[['id','ohx_status']].astype(int)
oral.iloc[:,2:] = oral.iloc[:,2:].astype('category')

pickle_out_demo = open("demo_file.pkl","wb")
pickle.dump(demo, pickle_out_demo)
pickle_out_demo.close()

pickle_in_demo = open("demo_file.pkl","rb")
example_demo = pickle.load(pickle_in_demo)
print(example_demo)

demo.to_pickle("demo_file.pkl")

#import pickle file
demo = pd.read_pickle("demo_file.pkl")

,id,age,race,education,martial_status,interview_status,maskedvar-psu,maskedvar-stra,fullsample2yr_mec,fullsample2yr_int,cohort
0,62161,22,3,3.0,5.0,2.0,1.0,91.0,NaN,102641.41,2011-2012
1,62162,3,1,NaN,NaN,2.0,3.0,92.0,NaN,15457.74,2011-2012
2,62163,14,6,NaN,NaN,2.0,3.0,90.0,NaN,7397.68,2011-2012
3,62164,44,3,4.0,1.0,2.0,1.0,94.0,NaN,127351.37,2011-2012
4,62165,14,4,NaN,NaN,2.0,2.0,90.0,NaN,12209.74,2011-2012
...,...,...,...,...,...,...,...,...,...,...,...
9249,102952,70,6,3.0,1.0,2.0,2.0,138.0,NaN,16896.28,2017-2018
9250,102953,42,1,3.0,4.0,2.0,2.0,137.0,NaN,61630.38,2017-2018
9251,102954,41,4,5.0,5.0,2.0,1.0,144.0,NaN,17160.90,2017-2018
9252,102955,14,4,NaN,NaN,2.0,1.0,136.0,NaN,14238.45,2017-2018


In [ ]:
#Merge demography and oralhealth by id
oral_selected = oral[['id','ohx_status']]
demo_merged = pd.merge(demo, oral_selected, on = 'id', how = 'left')

#Separated age into two levels: under 20 and above 20
demo_merged['under_20'] = np.where(demo_merged['age']<20, "True","False")

#Separate education into two levels: 'some college/college graduate', 'No college/<20'
conditions = [
    (demo_merged['education'] == 4.0) | (demo_merged['education'] == 5.0),
    (demo_merged['education'] == 1.0) | (demo_merged['education'] == 2.0) | (demo_merged['education'] == 3.0)]
choices = ['some college/college graduate', 'No college/<20']
demo_merged['college'] = np.select(conditions, choices, 'No college/<20')
demo_updated = demo_merged[['id','gender','age','under_20','college','exam_status','ohx_status']]

#Separate ohx into two levels: completing and missing
demo_updated['ohx'] = np.where((demo_updated['exam_status'] == 2.0)&(demo_updated['ohx_status'] ==1),'complete','missing')

#Replace categorical expression of gender
demo_updated['gender'] = np.where(demo_updated['gender'] == 1.0, 'Male', 'Female')
demo_updated

In [ ]:
#Keep only the exam_status == 2
demo_updated_ = demo_updated[demo_updated['exam_status'] == 2.0] 

#Compute the difference with previous data length and current data length
remains = demo_updated_['exam_status'].shape[0]
origins = demo_updated['exam_status'].shape[0]
diff = origins - remains

#Express through dictionary and dataFrame
data = {'rows_initial':origins, 'rows_remains':remains,'rows_removed':diff}
df = pd.DataFrame(data, index = ['num'])
df

In [ ]:
#Compute the count and percentage within categorical variable
summary = demo_updated_[['age','under_20','gender','college','ohx']]
summary['under_20'] = np.where(summary['under_20'] == 'False','above_20','under_20')
under_20_sum1 = pd.crosstab(summary['under_20'], summary['ohx'])
under_20_sum2 = pd.crosstab(summary['under_20'], summary['ohx'], normalize = 'index')
under_20 = (pd.concat([under_20_sum1, under_20_sum2], axis=1, keys=['count', 'pct%'])
      .swaplevel(axis=1)
      .sort_index(axis=1, ascending=[True, False])
)
gender_sum1 = pd.crosstab(summary['gender'], summary['ohx'])
gender_sum2 = pd.crosstab(summary['gender'], summary['ohx'], normalize = 'index')
gender = (pd.concat([gender_sum1, gender_sum2], axis=1, keys=['count', 'pct%'])
      .swaplevel(axis=1)
      .sort_index(axis=1, ascending=[True, False])
)
college_sum1 = pd.crosstab(summary['college'], summary['ohx'])
college_sum2 = pd.crosstab(summary['college'], summary['ohx'], normalize = 'index')
college = (pd.concat([college_sum1, college_sum2], axis=1, keys=['count', 'pct%'])
      .swaplevel(axis=1)
      .sort_index(axis=1, ascending=[True, False])
)
summ = pd.concat([under_20,gender,college])

In [ ]:
#Compute the mean and standard deviation within the continuous variable
age = summary.groupby('ohx').agg({'age': ['mean', 'std']}).transpose().reset_index()

In [ ]:
#Apply t-test towards age variable
ttest_ind_from_stats(mean1=age['complete'][0], std1=age['complete'][1], nobs1=13,
                     mean2=age['missing'][0], std2=age['missing'][1], nobs2=11)

In [ ]:
#Apply chi-square towards under_20 categorical variables
under_20_data = np.array([under_20.iloc[0][0:5].values,
                  under_20.iloc[1][0:5].values])
chi_20 = stats.chi2_contingency(under_20_data)[0:3]

In [ ]:
#Apply chi-square towards college categorical variables
college_data = np.array([college.iloc[0][0:5].values,
                  college.iloc[1][0:5].values])
chi_col = stats.chi2_contingency(college_data)[0:3]

In [ ]:
#Apply chi-square towards gender categorical variables
gender_data = np.array([gender.iloc[0][0:5].values,
                  gender.iloc[1][0:5].values])
chi_gen = stats.chi2_contingency(gender_data)[0:3]

In [ ]:
#Create Table
chi_data = {'under_20':chi_20, 'college':chi_col, 'gender':chi_gen}
chi_square = pd.DataFrame(chi_data, index = ['statistical_value', 'p-value', 'df'])
chi_square.iloc[:1] = chi_square.iloc[:1].applymap('{0:.2f}'.format)
chi_square